In [1]:
import shapefile

In [2]:
import os, os.path

In [3]:
from collections import namedtuple

NodeData = namedtuple( 'NodeData', [ 'level', 'x' ,'y' ] )
nodes = {}

SinkData = namedtuple( 'SinkData', [ 'level', 'building_name' ] )
sinks = {}

SourceData = namedtuple( 'SourceData', [ 'level', 'p_lim_kw', 'p_pa' ] )
sources = {}

PipeData = namedtuple( 'PipeData', [ 'length', 'diameter_m', 'from_node', 'to_node' ] )
pipes = {}

In [4]:
gas_nodes_data = shapefile.Reader( os.path.join( os.getcwd(), '..', 'data', 'shapefiles', 'gas_network_nodes' ) )
assert( gas_nodes_data.shapeType == shapefile.POINT )

In [5]:
for data in gas_nodes_data:
    x = data.shape.points[0][0]
    y = data.shape.points[0][1]
    nodes[data.record['name']] = NodeData( data.record['level'], x, y )
    if data.record['type'] == 'SINK':
        sinks[data.record['name']] = SinkData( data.record['level'], data.record['build_id'] )
    elif data.record['type'] == 'SRCE':
        sources[data.record['name']] = SourceData( data.record['level'], data.record['p_lim_kw'], data.record['p_lim_kw'] )

In [6]:
gas_pipes_data = shapefile.Reader( os.path.join( os.getcwd(), '..', 'data', 'shapefiles', 'gas_network_pipes' ) )
assert( gas_pipes_data.shapeType == shapefile.POLYLINE )

In [7]:
from math import *

for data in gas_pipes_data:
    p1 = data.shape.points[0]
    p2 = data.shape.points[1]
    length = sqrt( pow( p1[0] - p2[0], 2 ) + pow( p1[1] - p2[1], 2) )
    pipes[data.record['name']] = \
        PipeData( length, data.record['diameter_m'], data.record['from_node'], data.record['to_node'] )

In [8]:
from dblayer.helpers.utn.gas_network import *

connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
    )

db_access = DBAccess()
db_access.connect_to_citydb( connect )

In [9]:
srid = 4326

In [10]:
( ntw_id, ntw_graph_id ) = write_network_to_db(
    db_access,
    name = 'gas_network',
    id = 3000
    )

In [11]:
nodes_db_data = {}

In [12]:
for name, data in nodes.items():

    nodes_db_data[name] = write_network_node_to_db(
        db_access,
        name,
        data.level,
        Point2D( data.x, data.y ),
        srid,
        ntw_id,
        ntw_graph_id )

In [13]:
for name, data in pipes.items():

    write_round_pipe_to_db(
        db_access,
        name,
        nodes_db_data[data.from_node],
        nodes_db_data[data.to_node],
        srid,
        ntw_id,
        ntw_graph_id,
        int_diameter = data.diameter_m,
        int_diameter_unit = 'm'
    )

In [14]:
buildings_db_data = db_access.get_citydb_objects( 'Building', table_name='building', schema='citydb_view' )
buildings_db_id = { b.name: b.id for b in buildings_db_data}

In [15]:
for name, data in sinks.items():

    write_gas_sink_to_db(
        db_access,
        name,
        nodes_db_data[name],
        10.,
        'kW',
        srid,
        ntw_id,
        ntw_graph_id,
        buildings_db_id[data.building_name]
        )

In [16]:
for name, data in sources.items():

    write_feeder_to_db(
        db_access,
        name,
        nodes_db_data[name],
        data.p_lim_kw,
        data.p_pa,
        srid,
        ntw_id,
        ntw_graph_id
        )

In [17]:
db_access.commit_citydb_session()